In [1]:
pip install transformers --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
import torch
import numpy as np
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [6]:
test_text = pd.read_csv('ab_dev_reviews.txt', sep = '\t', header=None)
test_asp = pd.read_csv('dev_aspects.txt', sep = '\t', header=None)

In [7]:
res = [] # приводим данные к виду трейна
asp_vals = test_asp.values
for i in tqdm(range(len(test_asp))):
    
    id_ = asp_vals[i][0]
    text = test_text[test_text[0]==id_][1].values[0]
    
    w = asp_vals[i][2]
    qw = '@q@ ' + w + ' @q@'
    
    ln = [qw + ' ' + text]
    res.extend(ln)

100%|█████████████████████████████████████| 1190/1190 [00:00<00:00, 3522.30it/s]


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/ruBert-base')
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/ab_proj/checkpoint-1341').to('cuda')

Downloading:   0%|          | 0.00/590 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

In [7]:
inp = tokenizer(res, padding = 'max_length', max_length = 128, truncation=True, return_tensors='pt').to('cuda')

with torch.no_grad():
    logits = model(**inp).logits
    
out_df = pd.DataFrame(logits).astype('float')

In [8]:
maping = {0:'both',
          1: 'negative',
          2: 'neutral',
          3: 'positive'}

In [12]:
pred = [maping[np.argmax(i)] for i in out_df.values]
test = pd.read_csv('/content/drive/MyDrive/ab_proj/dev_aspects.txt', sep = '\t', header=None)

In [13]:
y_test = test[5].tolist()

In [14]:
from sklearn.metrics import accuracy_score
a = accuracy_score(pred, y_test)
print(f'aspect sentiment acc: {a}')

aspect sentiment acc: 0.9159663865546218
